## Using the Interpretune Neuronpedia Extension

This tutorial demonstrates Interpretune's integration with Circuit Tracer and Neuronpedia, guiding the user through generating and sharing a basic model attribution graph.
Future tutorials will demonstrate the use of Interpretune's analytical abstractions to
generate, transform and share more advanced model attribution graphs in the context of world model analysis, research or AI agent/application interpretability maximization.


In [ ]:
# this example assumes the user has either installed `interpretune` with the `examples` extra
# (or that circuit_tracer and neuronpedia are installed separately in addition to base interpretune dependencies)

# Import circuit tracer and required modules
from transformer_lens import ActivationCache  # noqa: F401
from datetime import datetime

import interpretune as it  # registered analysis ops will be available as it.<op> when analysis is imported
from it_examples import _ACTIVE_PATCHES # noqa: F401  # TODO: add note about this unless patched in SL before release
from it_examples.example_module_registry import MODULE_EXAMPLE_REGISTRY  # TODO: move to hub once implemented
from interpretune import ITSessionConfig, ITSession
from interpretune.base.call import it_init

# Note: You'll still need to set your API keys via environment variables:
# export NEURONPEDIA_API_KEY="your-production-key"
# export DEV_NEURONPEDIA_API_KEY="your-dev-key"  # if using dev API

In [ ]:
# Load our demo config (this will be done from the hub once that is available)
base_itdm_cfg, base_it_cfg, dm_cls, m_cls = MODULE_EXAMPLE_REGISTRY.get('gemma2.rte_demo.circuit_tracer_w_neuronpedia')

# configure our session with our desired adapter composition, core and circuit_tracer in this case
session_cfg = ITSessionConfig(adapter_ctx=(it.Adapter.core, it.Adapter.circuit_tracer), datamodule_cfg=base_itdm_cfg,
                              module_cfg=base_it_cfg, datamodule_cls=dm_cls, module_cls=m_cls)

# start our session
it_session = ITSession(session_cfg)

In [ ]:
# manual init for now
it_init(**it_session)
print("\nIT Session initialized successfully!")

### Example: Using Circuit Tracer to generate a custom graph and upload to Neuronpedia

In [ ]:
# Generate and upload graph in one step
prompt = "The capital of the state containing Dallas is"
ct_module = it_session.module
# we use the datetime to ensure the slug is unique
slug = f"it-generated-compute-specific-logits-demo-graph-{datetime.now().strftime('%Y%m%d-%H%M%S')}"
# This method does everything:
# - Generates attribution graph
# - Transforms it for Neuronpedia compatibility
# - Uploads to Neuronpedia
# - Returns both the graph and the Neuronpedia metadata

# if you want to set custom metadata for Neuronpedia graph, you can do so here.
# in this demo, since the circuit-tracer generated graph does not currently have cantor-paired feature nodes, we
# need to avoid setting neuronpedia_source_set in the feature_details metadata
# temporary hack: https://bit.ly/non_cantor_edge_case
graph, local_graph_path, neuronpedia_metadata = ct_module.generate_graph(
    prompt=prompt,
    slug=slug,
    upload_to_np=True,  # Upload the produced graph to Neuronpedia if True, or just return the graph if False
)

print (f"Generated and saved graph locally to {local_graph_path}")

if neuronpedia_metadata and hasattr(neuronpedia_metadata, 'url'):
    print("Graph uploaded successfully.")
    print(f"View your graph at: {neuronpedia_metadata.url}")

In [ ]:
# Alternative: Just transform an existing graph
# transformed_graph, np_metadata = ct_module.neuronpedia.transform_graph_for_np(
#     graph_path=local_graph_path,
#     # slug="it-generated-dallas-austin-demo-graph-20250725-121824-retransformed",
#     upload_to_np=False  # Set to True to upload the transformed graph
# )